In [1]:
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))


import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

from src.data_loader import get_data_generators
from src.model import build_model

In [2]:
DATA_DIR = '../data/raw'  
IMG_SIZE = (224, 224)     
BATCH_SIZE = 32           
EPOCHS = 30               

# Check if data directory exists
if not os.path.exists(DATA_DIR):
    print(f"Error: {DATA_DIR} not found. Please make sure you have downloaded the data!")
else:
    print(f"Data folder found: {DATA_DIR}")

Data folder found: ../data/raw


In [3]:
# Data Generators
print("\n--- Data loading ---")
try:
    train_gen, val_gen = get_data_generators(DATA_DIR, img_size=IMG_SIZE, batch_size=BATCH_SIZE)
    print(f"Found classes: {train_gen.num_classes}")
    print(f"Class names: {list(train_gen.class_indices.keys())}")
except Exception as e:
    print(f"Error loading data: {e}")


--- Data loading ---
Found 2039 images belonging to 11 classes.


Found 504 images belonging to 11 classes.


Found classes: 11
Class names: ['Ac', 'As', 'Cb', 'Cc', 'Ci', 'Cs', 'Ct', 'Cu', 'Ns', 'Sc', 'St']


In [4]:
# Build Model
print("\n--- Building model MobileNetV2 ---")
num_classes = train_gen.num_classes
model = build_model(num_classes=num_classes, input_shape=IMG_SIZE + (3,))
model.summary()


--- Building model MobileNetV2 ---


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,423,371 (9.24 MB)

 Trainable params: 165,387 (646.04 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [5]:
# Fine tuning setup
model.trainable = True

for layer in model.layers[:100]:
    layer.trainable = False


model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


print(f"Number of frozen layers: 100")
print(f"Number of trainable layers: {len(model.layers) - 100}")


Number of frozen layers: 100
Number of trainable layers: 57


In [6]:
print(f"Model gotowy do Fine-Tuningu. Liczba warstw do treningu: {len(model.layers) - 100}")

# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=1, min_lr=1e-6), 
    ModelCheckpoint('../models/cloud_model_best.h5', monitor='val_loss', save_best_only=True, verbose=1)
]

Model gotowy do Fine-Tuningu. Liczba warstw do treningu: 57


In [7]:
# Training
print(f"\n--- Training ({EPOCHS} epochs) ---")
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS,
    callbacks=callbacks
)

print("\n--- Training complete ---")


--- Training (30 epochs) ---


Epoch 1/30


 1/64 ━━━━━━━━━━━━━━━━━━━━ 1:35 2s/step - accuracy: 0.0938 - loss: 2.6476

 2/64 ━━━━━━━━━━━━━━━━━━━━ 9s 146ms/step - accuracy: 0.0859 - loss: 2.7210

 3/64 ━━━━━━━━━━━━━━━━━━━━ 8s 147ms/step - accuracy: 0.0781 - loss: 2.7407

...


Epoch 22: val_loss did not improve from 1.76615


64/64 ━━━━━━━━━━━━━━━━━━━━ 18s 275ms/step - accuracy: 0.5360 - loss: 1.4171 - val_accuracy: 0.3810 - val_loss: 1.8102 - learning_rate: 4.0000e-06


Epoch 22: early stopping


Restoring model weights from the end of the best epoch: 17.



--- Training complete ---


In [8]:
# Results 
print("\n--- Saving model ---")
if not os.path.exists('../models'):
    os.makedirs('../models')

save_path = '../models/cloud_model_best.h5'
model.save(save_path)
print(f"SUCCESS! Model saved at: {save_path}")


--- Saving model ---
SUCCESS! Model saved at: ../models/cloud_model_best.h5
